# Building an Operational Data Store with Firestore, Memorystore, and GCS

This notebook implements a multi-layer ODS approach using Google Cloud services. 

**Architecture (Google Native):**
1. **Operational Layer (Firestore):** Stores recent orders and serves individual document lookups.
2. **Aggregate Document Pattern (Firestore):** Provides pre-computed metrics for standard dashboards. Efficient and cost-effective.
3. **Speed Layer (Memorystore for Redis - Optional):** For extreme throughput and sub-millisecond latency. **Memorystore** is Google's managed version of Redis.
4. **Archive Layer (GCS):** Stores monthly exports for long-term durability and compliance.

## Part 0: Setup

Install necessary packages if not already present:

In [ ]:
%pip install google-cloud-firestore google-cloud-storage redis python-dotenv pandas numpy --quiet

### Imports

In [ ]:
import os
import json
import time
from datetime import datetime, timedelta
from pathlib import Path

import numpy as np
import pandas as pd
from google.cloud import firestore, storage
import redis
from dotenv import load_dotenv

print("Imports OK")

## Part 1: Credentials

Load credentials from the `.env` file.

In [ ]:
load_dotenv(Path("../.env"))

GCP_PROJECT_ID = os.getenv("GCP_PROJECT_ID")
GCS_BUCKET_NAME = os.getenv("GCS_BUCKET_NAME")

REDIS_HOST = os.getenv("REDIS_HOST")
REDIS_PORT = int(os.getenv("REDIS_PORT", 6379))
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD")

print(f"Project ID: {GCP_PROJECT_ID}")

## Part 2: Connect to Firestore and Memorystore

In [ ]:
# Connect to Firestore
db = firestore.Client(project=GCP_PROJECT_ID)
print("Connected to Firestore.")

# Connect to Memorystore (Redis)
r = None
if REDIS_HOST and REDIS_HOST != "YOUR_REDIS_HOST":
    try:
        r = redis.Redis(
            host=REDIS_HOST,
            port=REDIS_PORT,
            password=REDIS_PASSWORD,
            decode_responses=True,
        )
        r.ping()
        print("Connected to Memorystore (Redis).")
    except Exception as e:
        print(f"Memorystore connection failed: {e}. Caching section will be skipped.")

## Part 3: Load Data into Firestore

In [ ]:
np.random.seed(42)

CATEGORIES = ["footwear", "apparel", "accessories", "electronics"]
products_data = []
for i in range(1, 21):
    category = np.random.choice(CATEGORIES)
    products_data.append({
        "product_id": f"PROD_{i:03d}",
        "name": f"{category.title()} Item {i}",
        "category": category,
        "price": round(np.random.uniform(20, 300), 2),
    })

REGIONS = ["North", "South", "East", "West"]
STATUSES = ["pending", "shipped", "delivered", "returned"]
orders_data = []
start_date = datetime.now() - timedelta(days=35)

for i in range(1, 101):
    product = np.random.choice(products_data)
    quantity = np.random.randint(1, 4)
    order_date = start_date + timedelta(days=int(np.random.randint(0, 35)),
                                        hours=int(np.random.randint(0, 24)))
    orders_data.append({
        "order_id": f"ORD_{i:05d}",
        "customer_id": f"CUST_{np.random.randint(1, 101):04d}",
        "region": np.random.choice(REGIONS),
        "product_id": product["product_id"],
        "category": product["category"],
        "unit_price": product["price"],
        "quantity": quantity,
        "total": round(product["price"] * quantity, 2),
        "status": np.random.choice(STATUSES, p=[0.05, 0.15, 0.75, 0.05]),
        "created_at": order_date,
    })

# Batch Insert
batch = db.batch()
orders_ref = db.collection("orders")
for order in orders_data:
    doc_ref = orders_ref.document(order["order_id"])
    batch.set(doc_ref, order)
batch.commit()
print(f"Inserted {len(orders_data)} orders into Firestore.")

## Part 4: The 'Aggregate Document' Pattern

Maintain a pre-computed summary document in Firestore.

In [ ]:
def update_aggregate():
    cutoff = datetime.now() - timedelta(days=30)
    docs = db.collection("orders").where("created_at", ">=", cutoff).where("status", "!=", "returned").stream()
    
    summary = {}
    for doc in docs:
        d = doc.to_dict()
        cat = d["category"]
        if cat not in summary:
            summary[cat] = {"revenue": 0, "orders": 0}
        summary[cat]["revenue"] += d["total"]
        summary[cat]["orders"] += 1
        
    db.collection("aggregates").document("category_revenue").set({
        "last_updated": firestore.SERVER_TIMESTAMP,
        "data": summary
    })
    print("Aggregate document updated in Firestore.")
    return summary

current_summary = update_aggregate()

## Part 5: The Speed Layer (Memorystore - Optional)

For extreme performance, we can push the aggregate summary further out into **Memorystore (Redis)**.

In [ ]:
def sync_to_memorystore(summary_data):
    if r:
        # Cache for 1 hour (extreme speed layer)
        r.set("dashboard:summary", json.dumps(summary_data), ex=3600)
        print("Metrics synced to Google Cloud Memorystore.")
    else:
        print("Memorystore not connected. Skipping speed layer sync.")

sync_to_memorystore(current_summary)

## Part 6: Efficient Dashboard Loading

Try Memorystore first (0.5ms), fallback to Firestore Aggregate (10ms).

In [ ]:
def get_dashboard_data():
    # 1. Try Speed Layer (Memorystore)
    if r:
        cached = r.get("dashboard:summary")
        if cached:
            print("[MEMORYSTORE HIT]")
            return json.loads(cached)
    
    # 2. Fallback to Operational Data Store (Firestore Aggregate)
    print("[MEMORYSTORE MISS] Reading from Firestore Aggregate...")
    doc = db.collection("aggregates").document("category_revenue").get()
    if doc.exists:
        return doc.to_dict()["data"]
    
    return {}

data = get_dashboard_data()
for cat, m in data.items():
    print(f"  {cat:<15}  ${m['revenue']:>8,.2f}")

## Part 7: Archive to GCS

Export month-to-date orders to JSONL and upload.

In [ ]:
if GCS_BUCKET_NAME and GCS_BUCKET_NAME != "YOUR_BUCKET_NAME":
    bucket = storage.Client(project=GCP_PROJECT_ID).bucket(GCS_BUCKET_NAME)
    export_path = Path("./orders_archive.jsonl")
    
    month_start = datetime.now().replace(day=1, hour=0, minute=0, second=0, microsecond=0)
    docs = db.collection("orders").where("created_at", ">=", month_start).stream()
    
    with open(export_path, "w") as f:
        for doc in docs:
            order = doc.to_dict()
            order["created_at"] = order["created_at"].isoformat() 
            f.write(json.dumps(order) + "\n")
            
    gcs_path = f"shopstream/orders/{datetime.now().strftime('%Y/%m/%d')}/orders_archive.jsonl"
    bucket.blob(gcs_path).upload_from_filename(str(export_path))
    print(f"Archived to: gs://{GCS_BUCKET_NAME}/{gcs_path}")
else:
    print("GCS_BUCKET_NAME not set. Archiving skipped.")